In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import warnings
import csv
warnings.filterwarnings('ignore')

In [ ]:
import cv2
from PIL import Image

In [ ]:
train_image = []
train_annotation = []

In [ ]:
import glob
path = "/Users/shenx/292_project/drone_dataset_yolo/dataset_txt/*.jpg"
image_file = glob.glob(path,recursive = True)

In [ ]:
image_file

In [ ]:
for i in image_file:     #slicing for insufficient memory
    Load_image = Image.open(i).convert('RGB')
    nmpy_img = Load_image.resize((258,258))
    train_image.append(np.asarray(nmpy_img))
    
    seperate_path = i.split('.')
    seperate_path[1] = '.txt'
    
    with open(seperate_path[0]+seperate_path[1]) as f:
        lines = f.readlines()
        tmp_lst = lines[0].split(' ')
        #label = int(tmp_lst[0])
        startX = float(tmp_lst[1])
        startY = float(tmp_lst[2])
        endX = float(tmp_lst[3])
        endY = float(tmp_lst[4])
        train_annotation.append((startX, startY, endX, endY))

In [ ]:
plt.figure(figsize=(20, 20))


for i in range(28):
    ax = plt.subplot(7, 7, i + 1)
    plt.imshow(train_image[i].astype("uint8"))
    plt.axis("off")

In [ ]:
from sklearn.model_selection import train_test_split
data = np.array(train_image, dtype='float32') / 255.0   # memory not sufficient !!
targets = np.array(train_annotation, dtype='float32')

split = train_test_split(data, targets, test_size=0.10, random_state=42)

(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]

In [ ]:
trainImages

In [ ]:
data.shape, targets.shape

In [ ]:
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras import Model,Input

from keras.backend import sigmoid

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.layers import Activation
tf.keras.utils.get_custom_objects().update({'swish': Activation(swish)})

def build_model():
    vgg = tf.keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(258, 258, 3)))

    vgg.trainable = True

    flatten = vgg.output
    flatten = Flatten()(flatten)

    bboxHead = Dense(128, activation="swish")(flatten)
    bboxHead = Dense(64, activation="swish")(bboxHead)
    bboxHead = Dense(32, activation="swish")(bboxHead)
    bboxHead = Dense(4, activation="linear")(bboxHead)

    model = Model(inputs=vgg.input, outputs=bboxHead)

    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['accuracy'])

save_best = tf.keras.callbacks.ModelCheckpoint("Model_swish.h5",monitor='val_loss',save_best_only=True, verbose=1)

In [15]:
model.fit(trainImages, trainTargets, validation_split=0.2, batch_size= 16, epochs=50, verbose=1, callbacks=[save_best])

Epoch 1/50
62/62 [==============================] - ETA: 0s - loss: 1.4121 - accuracy: 0.5859
Epoch 1: val_loss improved from inf to 0.03432, saving model to Model_swish.h5
62/62 [==============================] - 446s 7s/step - loss: 1.4121 - accuracy: 0.5859 - val_loss: 0.0343 - val_accuracy: 0.6082
Epoch 2/50
62/62 [==============================] - ETA: 0s - loss: 0.0330 - accuracy: 0.6442
Epoch 2: val_loss did not improve from 0.03432
62/62 [==============================] - 422s 7s/step - loss: 0.0330 - accuracy: 0.6442 - val_loss: 0.0344 - val_accuracy: 0.6082
Epoch 3/50
62/62 [==============================] - ETA: 0s - loss: 0.0328 - accuracy: 0.6431
Epoch 3: val_loss did not improve from 0.03432
62/62 [==============================] - 421s 7s/step - loss: 0.0328 - accuracy: 0.6431 - val_loss: 0.0459 - val_accuracy: 0.6082
Epoch 4/50
62/62 [==============================] - ETA: 0s - loss: 0.0333 - accuracy: 0.6431
Epoch 4: val_loss improved from 0.03432 to 0.03047, saving mo

In [4]:
from keras.layers import Activation
model = tf.keras.models.load_model('./Model_swish.h5')

ValueError: Unknown activation function: 'Activation'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [18]:
Predict_output = model.predict(trainImages[:], verbose=1)
Predict_output

1/1 [==============================] - 1s 1s/step


array([[0.4804201 , 0.49889237, 0.98262006, 0.8350003 ],
       [0.5786794 , 0.41185555, 0.43588865, 0.21967953],
       [0.5247107 , 0.5341997 , 0.3947753 , 0.49453682],
       [0.46977252, 0.30339992, 0.42761502, 0.54596186],
       [0.5144788 , 0.48816526, 0.7825347 , 0.33435065],
       [0.5301355 , 0.43287697, 0.64705616, 0.7180704 ],
       [0.4925926 , 0.43358597, 0.02071334, 0.03879903],
       [0.2214485 , 0.43548682, 0.25128418, 0.42207468],
       [0.49050885, 0.40540493, 0.54274493, 0.34002036],
       [0.48559898, 0.5118698 , 0.9028483 , 0.84078854],
       [0.52187634, 0.51775855, 0.865823  , 0.7512323 ],
       [0.45326364, 0.44689855, 0.7137232 , 0.65269536],
       [0.659537  , 0.35659346, 0.40784624, 0.66335785],
       [0.44134596, 0.33949357, 0.46700323, 0.48376614],
       [0.51947254, 0.62489486, 0.12667577, 0.07924453],
       [0.52989805, 0.3245391 , 0.41818678, 0.40610164],
       [0.47718513, 0.25307536, 0.73366255, 0.44396162],
       [0.41340148, 0.33209744,

In [19]:
Actual_output = trainTargets[:]
Actual_output

array([[0.520417, 0.547619, 0.959167, 0.844444],
       [0.591   , 0.439   , 0.588   , 0.25    ],
       [0.442083, 0.594881, 0.4575  , 0.573034],
       [0.445312, 0.294565, 0.39375 , 0.558696],
       [0.51875 , 0.508333, 0.8125  , 0.363333],
       [0.60339 , 0.431429, 0.664407, 0.731429],
       [0.481   , 0.476   , 0.027   , 0.037   ],
       [0.316406, 0.45463 , 0.413021, 0.561111],
       [0.485   , 0.368   , 0.534   , 0.431   ],
       [0.493359, 0.556614, 0.910156, 0.826233],
       [0.536702, 0.512411, 0.884043, 0.851064],
       [0.484   , 0.492   , 0.762   , 0.71    ],
       [0.622066, 0.409624, 0.561815, 0.701878],
       [0.441111, 0.331915, 0.553333, 0.468085],
       [0.529   , 0.636   , 0.185   , 0.189   ],
       [0.521354, 0.334347, 0.402083, 0.443769],
       [0.496582, 0.260417, 0.819336, 0.427083],
       [0.396667, 0.287411, 0.148   , 0.142518],
       [0.494792, 0.374486, 0.65625 , 0.596708],
       [0.532227, 0.52691 , 0.935547, 0.838542]], dtype=float32)

In [20]:
with open('Model_swish_predict.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(Predict_output)
with open('Model_swish_actual.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(Actual_output)